In [ ]:
import ee
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import utils.image_processing as ip
import utils.land_coverage as lc

ee.Initialize()

In [ ]:
lat, lon = 44.302649, 5.505659
img_folder = 'data/S2A_MSIL2A_20200815T104031_N0214_R008_T31TFK_20200815T115046.SAFE/'

pixel_column, pixel_row = ip.get_fire_pixels(img_folder, lat, lon)

In [ ]:
diff = ip.plot_ndvi_difference('./output/', 'montguers')
plt.plot(pixel_column, pixel_row, 'ro', markersize=3)
plt.show()

In [ ]:
frag_count = 15
split = ip.split_image(diff, frag_count)

ip.plot_split_image(split, frag_count)
plt.tight_layout()
plt.show()

In [ ]:
fire = ip.merge_images(2, [split[(13, 12)], split[(14, 12)]])
ip.imshow(fire, 'Montguers Fire')
plt.show()

In [ ]:
thresholds = np.arange(0.1, 0.31, 0.02)
areas = []
for thr in thresholds:
    tmp = ip.threshold_filter(fire, thr)
    area = round(ip.calculate_area(tmp, diff) * 100, 4)
    areas.append(area)

In [ ]:
true_area = 289.39

plt.figure(figsize=(8, 6))
plt.plot(thresholds, areas)
plt.hlines(true_area, thresholds[0], thresholds[-1], colors='r', linestyles='dashed')
plt.xlabel('Threshold')
plt.ylabel('Burnt Area (ha)')
plt.title('Montguers Fire')
plt.legend(['Calculated', 'True Value'])
plt.show()

In [ ]:
from skimage.morphology import dilation, square

footprint = square(3)
dilated = dilation(tmp, footprint)
ip.plot_comparison(tmp, dilated, 'Dilation')

print(round(ip.calculate_area(dilated, diff) * 100, 4), 'ha')

In [ ]:
v1 = 9700
v2 = 10700
delta_v = v2 - v1
h1 = 8700
h2 = h1 + delta_v

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(diff)
plt.vlines(v1, 0, diff.shape[0], color='r', linestyle='dashed', linewidth=1)
plt.vlines(v2, 0, diff.shape[0], color='r', linestyle='dashed', linewidth=1)
plt.hlines(h1, 0, diff.shape[1], color='r', linestyle='dashed', linewidth=1)
plt.hlines(h2, 0, diff.shape[1], color='r', linestyle='dashed', linewidth=1)
plt.tight_layout()
plt.show()

In [ ]:
fire = diff[h1:h2, v1:v2]
plt.imshow(fire)
plt.axis('off')
plt.savefig('./output/fire_var.png')
plt.show()

In [ ]:
fire = ip.threshold_filter(fire, 0.15)
plt.imshow(fire)
plt.show()

In [ ]:
rand_image = lc.create_sample_coordinates(fire, 42, 0.1)
ip.imshow(rand_image, 'Sampled Coordinates')
plt.show()

In [ ]:
coordinates = lc.get_coordinates_from_pixels(rand_image, h1, v1, img_folder)
coordinates[:5]

In [ ]:
coordinates = pd.DataFrame(coordinates, columns=['latitude', 'longitude'])
coordinates.to_csv('data/coords_utm_montguers.csv', index=False)
coordinates.head()

In [ ]:
land_cover_data = pd.read_csv('data/MODIS_LandCover_Type1.csv')
land_cover_data

In [ ]:
samples = np.arange(50, 1001, 50)
lc.create_plots(samples, coordinates, 42, land_cover_data,
    fire_name='montguers',
    out_folder='output/pie_chart_montguers/',
    save_fig=True)

In [ ]:
lc.make_pie_chart_gif(
    'montguers', 'output/pie_chart_montguers/',
    save_all=True, duration=500, loop=0)

<img src="output/pie_chart_montguers/montguers.gif" width="750" align="center">